In [ ]:
import numpy as np
import random
from bs4 import BeautifulSoup
import pandas as pd
from pm4py.objects.log.util import dataframe_utils
from pm4py.objects.conversion.log import converter as log_converter
from pm4py.algo.discovery.inductive import algorithm as inductive_miner
from pm4py.visualization.process_tree import visualizer as pt_visualizer
from pm4py.visualization.dfg import visualizer as dfg_visualization
from pm4py.algo.discovery.dfg import algorithm as dfg_discovery
import matplotlib.pyplot as plt
import networkx as nx
from colour import Color
import copy
from collections import Counter

from networkx.drawing.nx_agraph import graphviz_layout, to_agraph
import pygraphviz as pgv

import json

import seaborn as sns

np.random.seed(42)
random.seed(42)
import pm4py

from collections import Counter
import subprocess

In [ ]:
# Load the log
from pm4py.objects.log.importer.xes import importer as xes_importer
log_application = xes_importer.apply('BPI Challenge 2017.xes')

# Investigate implications in the log
Certain events always surround other events or are always followed by other events.
We discover some connections to reduce the size of later modells.

In [ ]:
# A_complete is always preceeded by "W_call after offers"
for trace in log_application:
    for pos_index in range(len(trace)):
        pos = trace[pos_index]
        if "A_Complete" in pos["concept:name"]:
            if "W_Call after offers" not in trace[pos_index-1]["concept:name"]:
                for e in trace[pos_index-5:pos_index+5]:
                    print(e["concept:name"])
                print("not followed")
                assert(False)

In [ ]:
# "W_call after offers" has always "A_complete"
for trace in log_application:
    for pos_index in range(len(trace)):
        pos = trace[pos_index]
        if "W_call after offers" in pos["concept:name"]:
            index_inner = pos_index+1
            while trace[index_inner]["concept:name"] == "W_call after offers":
                index_inner += 1
            if "A_complete" not in trace[index_inner]["concept:name"]:
                for e in trace[index_inner-5:index_inner+5]:
                    print(e["concept:name"])
                print("not followed")
                assert(False)

In [ ]:
# A_Incomplete is always surrounded by "W_Call incomplete files"
for trace in log_application:
    for pos_index in range(len(trace)):
        pos = trace[pos_index]
        if "A_Incomplete" in pos["concept:name"]:
            if "W_Call incomplete files" not in trace[pos_index-1]["concept:name"] and "W_Call incomplete files" not in trace[pos_index+1]["concept:name"]:
                for e in trace[pos_index-5:pos_index+5]:
                    print(e["concept:name"])
                print("not followed")
                assert(False)

In [ ]:
#Create offer is always followed by "Created"
for trace in log_application:
    for pos_index in range(len(trace)):
        pos = trace[pos_index]
        if "Create Offer" in pos["concept:name"]:
            if "Created" not in trace[pos_index+1]["concept:name"]:
                print("not followed")

In [ ]:
# number of calls
for trace in log_application:
    for pos_index in range(len(trace)):
        pos = trace[pos_index]
        if "Create Offer" in pos["concept:name"]:
            if "Created" not in trace[pos_index+1]["concept:name"]:
                print("not followed")

In [ ]:
def contains(trace, element):
    for event in trace:
        if event['concept:name']==element:
            return True

# Preprocessing of the log
Several operations are performed:
- The log is split into two parts at the concept drift
- Call events ('W_Call incomplete files' and 'W_Call after offers') are accumulated and discretized by time. 
- - With a runtime of 60 - 600 sec. is the event considered as "SHORT"
- - between 10min and 4h as "LONG"
- - exceeding 4h as "SUPER LONG"
- Other workflow events are ignored
- Cancellation events after A_Pending are ignored
- 'O_Create Offer' is tagged (enumerted to consider how many offers were created)

In [ ]:
log = copy.deepcopy(log_application)
log_before = pm4py.filter_time_range(log, "2011-03-09 00:00:00", "2016-06-30 23:59:59", mode='traces_contained')
log_after = pm4py.filter_time_range(log, "2016-08-01 00:00:00", "2018-03-09 00:00:00", mode='traces_contained')
print(np.shape(log))
print(np.shape(log_before))
print(np.shape(log_after))

In [ ]:
# The log is filtered based on variations.
def filter_log(log):
    perc = 2/len(log)
    return pm4py.filter_variants_by_coverage_percentage(log, perc)

filtered_log_before = filter_log(log_before)
filtered_log_after = filter_log(log_after)
print(np.shape(filtered_log_before))
print(np.shape(filtered_log_after))

In [ ]:
TIME_DIFF = 0
MIN_SPEAKING_TIME = 60
SPEAK_THRESHOLD = 600
DAY_DIFFERENCE = 20
durations = {}
def construct_log(log):
    terminal_states = ['A_Cancelled COMPANY', 'A_Cancelled CUSTOMER', 'A_Pending', 'TIMEOUT']#, 'A_Denied']
    to_merge = ['W_Call incomplete files', 'W_Call after offers', 'W_Complete application', 'W_Validate application']
    log_activities = []
    for trace in log:
        current_trace = [trace[0]]
        for i in range(1,len(trace)):
            pos = trace[i]
            if "W_Call" in trace[i]['concept:name']:
                # search for closing event
                if pos['lifecycle:transition'] in ["start", "resume"]:
                    for inner_index in range(i+1, len(trace)):
                        inner_pos = trace[inner_index]
                        if pos['concept:name'] == inner_pos['concept:name']:
                            if inner_pos['lifecycle:transition'] in ["suspend", "complete"]:                 
                                duration = (inner_pos['time:timestamp']-pos['time:timestamp']).total_seconds()
                                if duration > MIN_SPEAKING_TIME:
                                    if duration > 10000:
                                        print(i, inner_index)
                                        print(pos)
                                        print(inner_pos)
                                        print(trace[i: inner_index+1])
                                        print("#########")
                                    if pos['concept:name'] in current_trace[-1]["concept:name"]:
                                        current_trace[-1]["duration"] += duration
                                    else:
                                        current_trace.append(pos)
                                        current_trace[-1]['duration'] = duration
                                    if current_trace[-1]["duration"] < SPEAK_THRESHOLD:
                                        current_trace[-1]['concept:name'] = pos['concept:name']+" SHORT"
                                    elif current_trace[-1]["duration"] < 14400:
                                        current_trace[-1]['concept:name'] = pos['concept:name']+" LONG"
                                    else:
                                        current_trace[-1]['concept:name'] = pos['concept:name']+" SUPER LONG"
                                    if pos['concept:name'] not in durations:
                                        durations[trace[i]['concept:name']] = []
                                    durations[trace[i]['concept:name']].append(duration)
                            break
            if "W_" in trace[i]['concept:name']:
                continue
            if trace[i]['concept:name'] in ["A_Created", "A_Complete", "A_Incomplete"]:
                continue
            if trace[i]['concept:name'] == "A_Cancelled":
                current_trace.append(pos)
                if (trace[i]['time:timestamp']-trace[i-1]['time:timestamp']).days >= DAY_DIFFERENCE:
                    current_trace[-1]['concept:name'] = "TIMEOUT"
                else:
                    current_trace[-1]['concept:name'] += " CUSTOMER"
                continue
            if "O_Created" == trace[i]['concept:name']:
                continue # merge create and created
            """
            if "A_Cancelled" == trace[i]['concept:name']:
                current_trace.append(pos)
                if (trace[i]['time:timestamp']-trace[i-1]['time:timestamp']).days >28:
                    current_trace[-1]['concept:name'] = current_trace[-1]['concept:name']+" AUTO"
                else:
                    current_trace[-1]['concept:name'] = current_trace[-1]['concept:name']+" USER"
            """
            if trace[i]['concept:name'] in terminal_states:
                current_trace.append(pos)
            else:
                if trace[i]['concept:name'] in to_merge and trace[i]['concept:name'] == trace[i-1]['concept:name']:
                    continue
                elif (trace[i]['time:timestamp']-trace[i-1]['time:timestamp']).total_seconds()>TIME_DIFF:
                    current_trace.append(pos)
        if "A_Pending" in [pos['concept:name'] for pos in current_trace]:
            if "O_Cancelled" in [pos['concept:name'] for pos in current_trace]:
                for pos1 in current_trace:
                    if 'O_Cancelled' in pos1['concept:name']:
                        current_trace.remove(pos1)
        intersection = [i for i in trace if i['concept:name'] in terminal_states]
        for state in terminal_states:
            indices = [i for i, x in enumerate(current_trace) if x['concept:name'] == state]
            if indices:
                current_trace = current_trace[:indices[0]+1]
        if intersection:
            log_activities.append(current_trace)
            
    duration_incomplete_list = []
    duration_offer_list = []
    for trace in log_activities:
        duration_incomplete = 0
        duration_offer = 0
        for pos in trace:
            if "W_Call incomplete files" in pos["concept:name"]:
                duration_incomplete += pos["duration"]
            if "W_Call after offers" in pos["concept:name"]:
                duration_offer += pos["duration"]
        duration_incomplete_list.append(duration_incomplete)
        duration_offer_list.append(duration_offer)
    plt.hist(duration_incomplete_list,100)
    plt.show()
    plt.hist(duration_offer_list,100)
    plt.show()
    
    print(len(durations))
    after_offers = durations["W_Call after offers SHORT"]+durations["W_Call after offers LONG"]
    incomplete_files = durations["W_Call incomplete files SHORT"]+durations["W_Call incomplete files LONG"]
    print(max(after_offers)) 
    print(max(incomplete_files))
    print("W_Call after offers")
    plt.hist(after_offers, 100)
    plt.show()
    print("W_Call incomplete files")
    plt.hist(incomplete_files, 100)
    plt.show()
    
    return log_activities

print(len(filtered_log_before), len(filtered_log_after))
filtered_log_before = construct_log(filtered_log_before)
filtered_log_after = construct_log(filtered_log_after)
print(len(filtered_log_before), len(filtered_log_after))

In [ ]:
# process log to iterate created offers
MAX_INDEX = 100
def unroll_log(log):
    
    for trace in log:
        #trace.insert(0,{'concept:name': "start"})
        isPositive = False
        if contains(trace, 'A_Pending'):
            isPositive = True
        trace.insert(0,{'concept:name': "start"})
        if isPositive:
            trace.append({'concept:name': "positive"})
        else:
            trace.append({'concept:name': "negative"})
    
    to_extend = ["O_Create Offer", "O_Created", "W_Call after offers", "W_Call incomplete files", "W_Validate application", "O_Sent (mail and online)", "O_Sent (online only)"]
    to_extend = ["O_Create Offer"]
    for name in to_extend:
        element = 0
        max_trace = 0
        max_index = 0
        for trace in log:
            indices = [i for i, x in enumerate(trace) if x['concept:name'] == name]
            for i in indices:
                count_indices = [j for j in indices if j < i]
                index = MAX_INDEX if len(count_indices) > MAX_INDEX else len(count_indices)
                trace[i]['concept:name'] += " "+str(index)
                if len(count_indices)>max_index:
                    max_index = len(count_indices)
                    element = trace[i]['concept:name']
                    max_trace = trace
        print(element, max_trace)
    return log

unroll_log(filtered_log_before)
unroll_log(filtered_log_after)

In [ ]:
def plot_lengths(log):
    lenghts = [len(x) for x in log]
    s = {}
    for e in lenghts:
        s[e] = lenghts.count(e)
    print(s)
    plt.hist(lenghts, len(set(lenghts)))
    plt.show()
plot_lengths(filtered_log_before)
plot_lengths(filtered_log_after)

In [ ]:
def plot_negative_positive(log):
    outcome = ["positive" if "positive" in [i['concept:name'] for i in trace] else "negative" for trace in log]
    print(Counter(outcome))
plot_negative_positive(filtered_log_before)
plot_negative_positive(filtered_log_after)

In [ ]:
REMOVE_THRESHOLD = 0

def ms(trace):
    multiset = {}
    for pos in trace:
        if pos['concept:name'] not in multiset:
            multiset[pos['concept:name']] = 1
        else:
            multiset[pos['concept:name']] += 1
    return json.dumps(multiset, sort_keys=True).encode()

def hist(trace): 
    hist = str(trace[0]['concept:name'])
    for pos in trace[1:]:
        hist += " - " + str(pos['concept:name'])
    return hist

def transition_system(log, threshold = 0, history = 1, actors = {}):
    edges = []
    edge_counter = {}
    controll = {}
    action = {}
    edge_mapping = {}
    for trace_index in range(len(log)):
        trace = log[trace_index]
        s = "start"
        assert(trace[0]['concept:name']=="start")
        for pos_index in range(1,len(trace)):
            pos = trace[pos_index]
            activity = pos['concept:name']
            controllable = True
            for key in actors:
                if key in activity:
                    controllable = actors[key]=='company'
            #t = ms(trace[max(0,pos_index-history+1):pos_index+1])
            t = hist(trace[max(0,pos_index-history+1):pos_index+1])
            e = (s,t)
            controll[e] = controllable
            action[e] = activity
            if e not in edges:
                edges.append(e)
                edge_counter[e] = 1
                edge_mapping[e] = [trace_index]
            else:
                edge_counter[e] = edge_counter[e]+1
                edge_mapping[e].append(trace_index)
            s = t
    g = nx.DiGraph()
    for e in edges:
        g.add_edge(e[0], e[1])
    to_remove = []
    trace_index_to_remove = []
    for e in g.edges:
        if e[0] == e[1]:
            to_remove.append(e)
        if edge_counter[e] <= threshold:
            for trace_index in edge_mapping[e]:
                trace_index_to_remove.append(trace_index)
        g[e[0]][e[1]]['controllable'] = controll[e]
        g[e[0]][e[1]]['action'] = action[e]
        if not controll[e]:
            g[e[0]][e[1]]['prob_weight'] = len(edge_mapping[e])

    for e in to_remove:
        if e in g.edges():
            g.remove_edge(e[0],e[1])
    print("numer of removed index", len(trace_index_to_remove))
    
    log_reduced = copy.deepcopy(log)
    for index in trace_index_to_remove:
        log_reduced.pop(index)
    
    if threshold>0:
        return transition_system(log_reduced, history = history, actors = actors)
    
    return g, edge_mapping

with open('activities.xml') as f:
    data = f.read()
actors = json.loads(data)

system_before, edge_mapping_before = transition_system(filtered_log_before, REMOVE_THRESHOLD, history = 3, actors = actors)
system_after, edge_mapping_after = transition_system(filtered_log_after, REMOVE_THRESHOLD, history = 3, actors = actors)

In [ ]:
# compute weights
def isInTrace(s,t, trace):
    for i in range(len(trace)-1):
        if trace[i]['concept:name'] == s and trace[i+1]['concept:name'] == t:
            return True
    return False

def weight(trace):
    return 1 if any("positive" in pos['concept:name'] for pos in trace) else -1

def entropy(p1, p2):
    if p1 == 0 or p2 == 0:
        return 0
    return - p1*np.log2(p1) - p2* np.log2(p2)

def distribution(s,t,log, edge_mapping):
    distr = {1.0: 0 , -1.0 : 0}
    assert((s,t) in edge_mapping)
    for trace_index in edge_mapping[(s,t)]:
        w = weight(log[trace_index])
        distr[w] += 1 #
    return distr[1], distr[-1]

def compute_edge_cost(g, traces, edge_mapping):
    #weights = [1 if "finished" in i else -1 for i in traces]
    edge_cost = {}
    counter = 1
    for s in g.nodes:
        counter +=1
        for t in g[s]:
            #if s == "start" or s == "fin" or t == "start" or t == "fin":
                #continue
            #print("s:", s, "t:",t)
            
            p1, p2 = distribution(s,t,traces, edge_mapping)
            w = 1 if p1 >= p2 else -1
            #print("distribution", p1, p2)
            wp1 = p1/(p1+p2)
            wp2 = p2/(p1+p2)
            #w = majority(s,t,traces)
            scaling = 10
            entro = entropy(wp1, wp2)
            #print("entropy",wp1, wp2,entro)
            edge_cost[(s,t)] = (((1-entro) * w) -0.21 )*20
            #print(edge_cost[(s,t)])
    return edge_cost

edge_cost_before = compute_edge_cost(system_before, filtered_log_before, edge_mapping_before)
edge_cost_after = compute_edge_cost(system_after, filtered_log_after, edge_mapping_after)

In [ ]:
def draw_dfg(g, name, layout = "sfdp"):
    # build graph with variable thicknes
    #scaling = 1/np.mean(list(nx.get_edge_attributes(g,'edge_weight').values()))

    A = to_agraph(g)
    edge_weights = nx.get_edge_attributes(g,'edge_weight')
    for e in edge_weights:
        e = A.get_edge(e[0], e[1])
        e.attr["penwidth"] = edge_weights[e]*scaling
        e.attr["fontsize"] = "20"
    for e in g.edges:
        if 'controllable' in g[e[0]][e[1]]:
            if not g[e[0]][e[1]]['controllable']:
                edge = A.get_edge(e[0], e[1])
                edge.attr["style"] = "dotted"
                edge.attr["label"] += " count " + str(g[e[0]][e[1]]["prob_weight"])
        #A.add_edge(e[0], e[1], penwidth = edge_weights[e]*scaling)

    #A.graph_attr.update(size="7.75,10.25")
    A.write(name.split(".")[0]+".dot")
    A.layout(layout)
    print("Plotted", name)
    A.draw(name)
    
def annotate_graph(g, edge_cost):
    for e in edge_cost:
        g[e[0]][e[1]]['label'] = round(edge_cost[e],2)
    return g

def color_graph(g, green = 4, red = -4):
    g = copy.deepcopy(g)
    for e in g.edges:
        if g[e[0]][e[1]]['label'] > green:
            g[e[0]][e[1]]['color'] ="green"
        elif g[e[0]][e[1]]['label'] < red:
            g[e[0]][e[1]]['color'] ="red"
        else:
            g[e[0]][e[1]]['color'] ="gray"
    
    return g

g_before = annotate_graph(system_before, edge_cost_before)
g_before_colored = color_graph(g_before)
draw_dfg(g_before, "graph_weight_before.ps", "dot")
draw_dfg(g_before_colored, 'colored_before.ps', "dot")

g_after = annotate_graph(system_after, edge_cost_after)
g_after_colored = color_graph(g_after)
draw_dfg(g_after, "graph_weight_after.ps", "dot")
draw_dfg(g_after_colored, 'colored_after.ps', "dot")

In [ ]:
def replay(g, trace, debug = False):
    count = 0
    s = "start"
    for pos_index in range(1,len(trace)):
        action = trace[pos_index]['concept:name']
        t = -1
        for n in g[s]:
            if g[s][n]['action'] == action:
                t = n
        if (s,t) not in g.edges:
            continue
        if debug:
            print(g[s][t]['label'])
        count += g[s][t]['label']
        s = t
    return count

def replay_traces(traces, g, name, debug = False):
    weights = np.array([weight(trace) for trace in traces])
    results = np.array([replay(g,trace) for trace in traces])
    labels = np.array(["true" if w == 1 else "false" for w in weights])
    if debug:
        for i in range(len(results)):
            print("State", weights[i], "has result: ",results[i])
    traces = np.array(traces)
    if debug:
        print(np.array(weights) == 1)
    m = weights == 1

    plt.scatter(np.array(range(len(results)))[m],results[m],c = "g", label="Successfull", s = 2)
    plt.scatter(np.array(range(len(results)))[~m],results[~m],c = "r", label="Unsuccessfull", s = 2)
    plt.title("Gas per Journey")
    plt.xlabel("Journey")
    plt.ylabel("Gas")
    plt.legend()
    #plt.xticks(np.arange(0,len(results),5))
    plt.savefig(name)
    plt.show()

    traces_length = np.array([len(i) for i in traces])
    plt.scatter(np.array(range(len(results)))[m],traces_length[m],c = "g", label="Successfull", s = 2)
    plt.scatter(np.array(range(len(results)))[~m],traces_length[~m],c = "r", label="Unsuccessfull", s = 2)
    plt.title("Length of traces")
    plt.show()

replay_traces(filtered_log_before, g_before, "traces_train_before.png", debug = False)
replay_traces(filtered_log_after, g_after, "traces_train_after.png", debug = False)

In [ ]:
def print_cycles(g):
    for c in nx.simple_cycles(g):
        count = 0
        c.append(c[0])
        for i in range(len(c)-1):
            count += g[c[i]][c[i+1]]['label']
        print(len(c))
        print("count", count)
print_cycles(g_before)
print("########")
print_cycles(g_after)

In [ ]:
def investigate_calls(log):
    for element in ["W_Call after offers", "W_Call incomplete files"]:
        # claim: calls are not really helpful...
        outcomes_call = {"pos": 0, "neg": 0}
        outcomes_no_call = {"pos": 0, "neg": 0}
        outcomes_short = {"pos": 0, "neg": 0}
        outcomes_long = {"pos": 0, "neg": 0}
        outcomes_super = {"pos": 0, "neg": 0}
        for trace in log:
            call = [element in pos["concept:name"] for pos in trace]
            short = [element+" SHORT" in pos["concept:name"] for pos in trace]
            long = [element+" LONG" in pos["concept:name"] for pos in trace]
            super_long = [element+" SUPER LONG" in pos["concept:name"] for pos in trace]
            result = ["positive" in pos["concept:name"] for pos in trace]
            if sum(call) > 0:
                if sum(result) > 0:
                    outcomes_call["pos"] += 1
                    if sum(short) > 0:
                        outcomes_short["pos"] += 1
                    elif sum(long) > 0:
                        outcomes_long["pos"] += 1
                    elif sum(super_long) > 0:
                         outcomes_super["pos"] += 1
                else:
                    outcomes_call["neg"] += 1
                    if sum(short) > 0:
                        outcomes_short["neg"] += 1
                    elif sum(long) > 0:
                        outcomes_long["neg"] += 1
                    elif sum(super_long) > 0:
                         outcomes_super["neg"] += 1
            else:
                if sum(result) > 0:
                    outcomes_no_call["pos"] += 1
                else:
                    outcomes_no_call["neg"] += 1
        
        print(element)
        print("no call")
        print(outcomes_no_call)
        print(outcomes_no_call["pos"]/(outcomes_no_call["neg"]+outcomes_no_call["pos"]))
        print("call")
        print(outcomes_call)
        print(outcomes_call["pos"]/(outcomes_call["neg"]+outcomes_call["pos"]))
        print("short")
        print(outcomes_short)
        print(outcomes_short["pos"]/(outcomes_short["neg"]+outcomes_short["pos"]))
        print("long")
        print(outcomes_long)
        print(outcomes_long["pos"]/(outcomes_long["neg"]+outcomes_long["pos"]))
        print("super")
        print(outcomes_super)
        print(outcomes_super["pos"]/(outcomes_super["neg"]+outcomes_super["pos"]))
        print()
        # calls not really impact
        # after drift are super long calls 3% more effective (0.34)
    
investigate_calls(filtered_log_before)
print("########")
investigate_calls(filtered_log_after)

In [ ]:
def compute_bounds(g, start, depth):
    lower_bounds = {}
    upper_bounds = {}
    queue = [(start,0)]
    counter = 0
    while(queue and counter < depth):
        counter += 1
        s, previous_gas = queue.pop(0)
        for t in g[s]:
            current_gas = previous_gas + g[s][t]['label']
            if t not in lower_bounds:
                lower_bounds[t] = current_gas
            else:
                lower_bounds[t] = min(lower_bounds[t], current_gas)
            if t not in upper_bounds:
                upper_bounds[t] = current_gas
            else:
                upper_bounds[t] = max(upper_bounds[t], current_gas)
            queue.append((t, current_gas))
    return lower_bounds, upper_bounds
lower_bounds_before, upper_bounds_before = compute_bounds(g_before_colored, "start", 200)
lower_bounds_after, upper_bounds_after = compute_bounds(g_after_colored, "start", 200)

In [ ]:
def call_responses(lower_bounds, upper_bounds):
    responses_lower = []
    responses_upper = []
    states = []
    names = []
    for element in lower_bounds:
        current_lower = []
        current_upper = []
        current_names = []
        found = False
        if "SHORT" in element.split("-")[-2] or "SHORT" in element.split("-")[0]:
            short = element
            long = element.replace("SHORT", "LONG")
            very = element.replace("SHORT", "SUPER LONG")
            found = True
        """
        elif "LONG" in element.split("-")[-2] or "LONG" in element.split("-")[0]:
            long = element
            short = element.replace("LONG", "SHORT")
            very = element.replace("LONG", "SUPER LONG")
            found = True
        """

        if not found:
            continue
        if short in lower_bounds:
            print("short", lower_bounds[short])
            current_lower.append(lower_bounds[short])
            current_upper.append(upper_bounds[short])
            current_names.append(short)
        else:
            current_lower.append(None)
            current_upper.append(None)
            current_names.append(None)
        if long in lower_bounds:
            print("long", lower_bounds[long])
            current_lower.append(lower_bounds[long])
            current_upper.append(upper_bounds[long])
            current_names.append(long)
        else:
            current_lower.append(None)
            current_upper.append(None)
            current_names.append(None)
        if very in lower_bounds:
            print("very", lower_bounds[very])
            current_lower.append(lower_bounds[very])
            current_upper.append(upper_bounds[very])
            current_names.append(very)
        else:
            current_lower.append(None)
            current_upper.append(None)
            current_names.append(None)
        if current_lower.count(None)==0:
            states.append(short)
            responses_lower.append(current_lower)
            responses_upper.append(current_upper)
            names.append(current_names)
            print("added")
        print("")
    print(states)
    print(len(states))
    return responses_lower, responses_upper, names

def plot_bounds(lower, upper, names, plot_name):
    colors = ["r", "y", "g"]
    for element in range(len(call_lower_before)):
        for index in range(len(call_lower_before[element])):
            if call_upper_before[element][index] == None:
                assert(False)
                continue
            height = upper[element][index] - lower[element][index]
            print(height)
            plt.bar(element+index*0.2, height = height, width = 0.2, color = colors[index], bottom = lower[element][index])
            print(names[element][index])
    plt.savefig(plot_name)
    plt.xticks(np.arange(len(lower)),labels = [name_dict[n[0]] for n in names])
    plt.show()
    print(len(names))

call_lower_before, call_upper_before, names_before = call_responses(lower_bounds_before, upper_bounds_before)
print("######")
call_lower_after, call_upper_after, names_after = call_responses(lower_bounds_after, upper_bounds_after)

name_dict = {}
count = 0
for n in names_before:
    if n[0] not in name_dict:
        name_dict[n[0]] = "S"+str(count)
        count += 1
for n in names_after:
    if n[0] not in name_dict:
        name_dict[n[0]] = "S"+str(count)
        count += 1

plot_bounds(call_lower_before, call_upper_before, names_before, "calls_before.png")
plot_bounds(call_lower_after, call_upper_after, names_after, "calls_after.png")

#plt.bar([1,2,3,4], height=[2,2.5,5,3], bottom=[0,-1,1,2])


In [ ]:
# Computes all possible shift of lists
def shifted_lists(l):
    shifted_lists = []
    for j in range(len(l)):
        list_constructed = copy.deepcopy(l[j:])
        list_constructed.extend(l[:j])
        list_constructed.append(list_constructed[0])
        shifted_lists.append(list_constructed)
    return shifted_lists

# checks if history hist contains circle c
def contains(hist, c):
    n = len(c)+1
    max_count = 0
    lists = shifted_lists(c)
    for helper_list in lists:
        count = 0
        for i in range(len(hist)-(n-1)):
            if hist[i:i+n] == helper_list:
                count += 1
        max_count = max(max_count, count)
    return max_count

# returns true if edge (e,v) is on c
def is_on(e,v,c):
    for i in range(len(c)-1):
        if c[i] == e and c[i+1] == v:
            return True
    if c[-1] == e and c[0] == v:
        return True
    
# Presented Unrolling algorithm, Algorithm 1 with online reducing
def unroll(G, start, target, k, debug = False):
    G_gen = nx.DiGraph()
    G_gen.add_node(start, hist = [str(start)])
    if 'controllable' in G.nodes[start]:
        G_gen.nodes[start]["controllable"] = G.nodes[start]["controllable"]

    cycles = list(nx.simple_cycles(G))

    queue = [start]
    # start bf-search
    while(queue):
        if debug:
            print(len(G_gen.nodes), len(queue))
        s = queue[0]
        queue.pop(0)
        s_original = str(s).split(".")[0]
        neighbours = list(G[s_original])
        for t in neighbours:
            t_original = t
            local_hist = copy.deepcopy(G_gen.nodes[s]["hist"])
            local_hist.append(str(t_original))
            is_on_cycle = False
            can_traverse = False
            path = []
            circle = []
            relevant_cycle = []
            for c in cycles:
                if is_on(s_original,t_original,c):
                    relevant_cycle.append(c)
                    
            all_smaller = True
            for c in relevant_cycle:
                if contains(local_hist,c) >= k:
                    all_smaller = False
            
            if not all_smaller:
                paths = list(nx.all_simple_paths(G, source=t, target=target))
                for p in paths:
                    merged_hist = copy.deepcopy(local_hist)
                    merged_hist.extend(p[1:]) # 1.st element already added
                    can_not_traverse = False
                    
                    #test if no loop larger than k with path
                    for c_loop in relevant_cycle:
                        if contains(merged_hist,c_loop) > k : # check that there is path without completing additional cycle
                            can_not_traverse = True
                    can_traverse = not can_not_traverse
            if all_smaller or can_traverse:               
                #every node not on cycle can be unqiue ("merge point" within unrolled graph)
                if relevant_cycle:
                    while t in G_gen.nodes:
                        if "." not in t:
                            t += ".1"
                        else:
                            t = t.split(".")[0]+"."+str(int(t.split(".")[-1])+1)
                # add node t only to graph if not already treated

                if t not in queue:
                    queue.append(t)
                    G_gen.add_node(t, hist = local_hist)
                assert(s in G_gen and t in G_gen)
                G_gen.add_edge(s,t)
                if('label' in G[s_original][t_original]):
                    G_gen[s][t]['label'] = G[s_original][t_original]['label']
                if('controllable' in G[s_original][t_original]):
                    G_gen[s][t]['controllable'] = G[s_original][t_original]['controllable']
                if('prob_weight' in G[s_original][t_original]):
                    G_gen[s][t]['prob_weight'] = G[s_original][t_original]['prob_weight']

    print("Graph was unrolled")
    return G_gen

In [ ]:
target = [s for s in g_before_colored.nodes if "positive" in s or "negative" in s]
g_before_unroll = unroll(g_before_colored, "start", target, 1)
target = [s for s in g_after_colored.nodes if "positive" in s or "negative" in s]
g_after_unroll = unroll(g_after_colored, "start", target, 1)

In [ ]:
def final_sound(g):
    # positive final states (should be one)
    positives = []
    for s in g.nodes:
        if "positive" in s:
            positives.append(s)
            print(s)
    assert(len(positives) == 1)
    
final_sound(g_before_unroll)
final_sound(g_after_unroll)

In [ ]:
out_of_gas = False
def to_uppaal(g, name, layout = "sfdp"):
    f = open("/home/paul/Downloads/uppaal-4.1.20-stratego-9-linux64/"+name, "w+")
    
    pos = nx.drawing.nx_agraph.graphviz_layout(g, prog=layout, args='-Grankdir=LR')

    f.write('<?xml version="1.0" encoding="utf-8"?>')
    f.write("<!DOCTYPE nta PUBLIC '-//Uppaal Team//DTD Flat System 1.1//EN' 'http://www.it.uu.se/research/group/darts/uppaal/flat-1_1.dtd'>")
    f.write('<nta>')
    f.write('<declaration>')
    f.write('int e = 0;')
    f.write('\n'+'clock x;')
    f.write('\n'+'hybrid clock t;')
    f.write('\n'+'int steps;')
    f.write('\n'+'bool negative = false;')
    f.write('\n'+'bool positive = false;')
    f.write('\n'+'int final_gas = -1;')
    f.write('</declaration>')
    f.write('<template>')
    f.write('<name x="5" y="5">Template</name>')
    
    # print locations
    ids = {}
    branch = {}
    for s,i in zip(pos, range(len(pos))):
        ids[s] = i
        print_location(f, "id"+str(i),pos[s][0],pos[s][1],s)
        f.write('\n')

    i = max(ids.values())+1
    
    for s in pos:
        for e in g.edges(s):
            if "controllable" in g[e[0]][e[1]]:
                if not g[e[0]][e[1]]["controllable"]:
                    f.write('<branchpoint id="id'+str(i)+'" x="'+str(int(pos[s][0])+20)+'" y="'+str(int(pos[s][1])+20)+'"/>')
                    f.write("\n")
                    branch[s]=i
                    i+=1
                    break # print branchpoint only once per state
                    
    f.write('<init ref="id'+str(ids['start'])+'"/>')
    
    for s in pos:
        for e in g.edges(s):
            if "controllable" in g[e[0]][e[1]]:
                if not g[e[0]][e[1]]["controllable"]:
                    f.write('<transition id="'+str(i)+'" controllable="false" >')
                    i = i+1
                    f.write('<source ref="id'+str(ids[s])+'"/>')
                    f.write('<target ref="id'+str(branch[s])+'"/>')
                    f.write('<label kind="assignment" x="'+str(int(pos[s][0]))+'" y="'+str(int(pos[s][1]))+'">')
                    f.write(' steps += 1')
                    f.write(',\n'+ 'x = 0')
                    f.write('</label>')
                    #f.write('<label kind="guard" x="'+str(int(pos[s][0]))+'" y="'+str(int(pos[s][0]))+'"> x&gt;= 1')
                    #f.write('</label>')
                    f.write('</transition>')
                    f.write("\n")
                    break # print branchpoint-edge only once per state
    
    max_id = max(ids.values())
    ids["outOfGas"] = max_id+1
    
    if out_of_gas:
        print_location(f,"id"+str(ids["outOfGas"]),-100,-100,"outOfGas")
    
    for e in g.edges:
        if "label" in g[e[0]][e[1]] and "controllable" in g[e[0]][e[1]]:
            if not g[e[0]][e[1]]["controllable"]:
                print_edge(f, branch[e[0]], ids[e[1]], pos[e[0]], pos[e[1]], g[e[0]][e[1]]['label'], g[e[0]][e[1]]['controllable'], e, g)
            else:
                print_edge(f, ids[e[0]], ids[e[1]], pos[e[0]], pos[e[1]], g[e[0]][e[1]]['label'], g[e[0]][e[1]]['controllable'], e, g)
        else:
            if "label" not in g[e[0]][e[1]]:
                if "controllable" not in g[e[0]][e[1]]:
                    print_edge(f, ids[e[0]], ids[e[1]], pos[e[0]], pos[e[1]], 0, True, e, g)

    if out_of_gas:
        for s in g.nodes:
            print(s)
            if "positive" in str(s) or "negative" in str(s):
                continue
            print(ids[s], ids["outOfGas"])
            print_edge(f, ids[s], ids["outOfGas"], pos[s], (-100,-100), 0,True, (s,"outOfGas"), g, guard = False)
    f.write('</template>')
    f.write('<system>')
    f.write('Journey = Template();')
    f.write('system Journey;')
    f.write('</system>')
    f.write('</nta>')
    f.close()
    print("all written to", f.name)

def print_location(f, location_id, x, y, name):
    name = str(name)
    name = name.replace('"', '-')
    name = name.replace('{', '')
    name = name.replace('}', '')
    name = name.replace("'", '-')
    name = name.replace("_", '')
    name = name.replace("(", '')
    name = name.replace(")", '')
    f.write('<location id="'+location_id+'" x="'+str(int(x))+'" y="'+str(int(y))+'">')
    f.write('<name x="'+str(int(x))+'" y="'+str(int(y)+20)+'">'+str(name).replace(":", "").replace(" ","").replace(".", "").replace(",", "").replace("-","")+'</name>')
    f.write('<label kind="invariant" x="'+str(int(x))+'" y="'+str(int(y)-30)+'">')
    if "positive" not in name and "negative" not in name and "outOfGas" not in name:
        f.write('x &lt;= ' + str(2))
    else:
        f.write("t'==0")
    f.write('</label>')
    f.write('</location>')

def print_edge(f, s, t, pos_s, pos_t, w, controllable, e, g, guard = False):
    x = (pos_s[0]+pos_t[0])/2
    y = (pos_s[1]+pos_t[1])/2
    if controllable:
        f.write('<transition action = "">')
    else:
        f.write('<transition controllable="false" action = "">')
    f.write('<source ref="id'+str(s)+'"/>')
    f.write('<target ref="id'+str(t)+'"/>')
    
    if controllable and False:
        f.write('<label kind="guard" x="'+str(int(x))+'" y="'+str(int(y))+'"> x&gt;= 1')
        f.write('</label>')
        
    if out_of_gas:
        if guard:
            f.write('<label kind="guard" x="'+str(int(x))+'" y="'+str(int(y))+'"> e&gt;= 0')
            f.write('</label>')
        else:
            f.write('<label kind="guard" x="'+str(int(x))+'" y="'+str(int(y))+'"> e&lt; 0')
            f.write('</label>')
        
    #f.write(',\n'+ 'x = 0')
    f.write('<label kind="assignment" x="'+str(int(x))+'" y="'+str(int(y))+'">')
    f.write(' steps += 1')
    f.write(',\n'+ 'x = 0')
    if "positive" in str(e[1]):
        f.write(',\n'+ 'positive = true')
        f.write(',\n'+ 'final_gas = e +'+str(int(round(w))))
    elif "negative" in str(e[1]):
        f.write(',\n'+ 'negative = true')
        f.write(',\n'+ 'final_gas = e + '+str(int(round(w))))
        
    f.write(',\n'+'e = e + '+str(int(round(w))))
    f.write('</label>')
    
    if not controllable:
        f.write('<label kind="probability" x="'+str(int(x)-20)+'" y="'+str(int(y)-20)+'">'+str(int(g[e[0]][e[1]]['prob_weight'])))
        f.write('</label>')
    f.write('</transition>')

to_uppaal(g_before_colored, "bpi2017_before.xml", layout = "dot")
to_uppaal(g_before_unroll, "bpi2017_before_unroll.xml")

to_uppaal(g_after_colored, "bpi2017_after.xml", layout = "dot")
to_uppaal(g_after_unroll, "bpi2017_after_unroll.xml")

In [ ]:
# insertions can force an edge_count of 1
for e in edge_mapping_before:
    if len(edge_mapping_before[e])==1:
        assert("Call" in e[1] or "TIMEOUT" in e[1] or "CUSTOMER" in e[1] or "Create Offer")

In [ ]:
def to_prism(g, name, terminal, mapping):
    f = open("/home/paul/Downloads/prism-games-3.0-linux64/generated/"+name, "w+")
    f.write('smg \n')
    f.write('player P1 providerModule endplayer \n')
    f.write('player P2 userModule endplayer \n')
    
    # write global variables
    
    f.write('global gas : [-10000..10000] init 0; \n')
    f.write('global state : [0..100] init 0; \n')
    f.write('global user : bool; \n')
    f.write('global provider : bool; \n')
    
    f.write('global negative : bool; \n')
    f.write('global positive : bool; \n')
    f.write('global final_gas : [-1000..1000] init 0; \n')
    f.write('global steps : [0..1000] init 0; \n')
    
    f.write("const double q = 1/2; \n")
    
    states = {}
    for n,i in zip(list(g.nodes),range(len(g.nodes))):
        states[n] = str(i)
        
    edges_user = []
    edges_provider = []
    
    user_control = {}
    provider_control = {}
    
    for e in g.edges:
        if 'controllable' in g[e[0]][e[1]] and not g[e[0]][e[1]]["controllable"]:
            edges_user.append(e)
            user_control[str(e[0])] = True
        else:
            edges_provider.append(e)
            provider_control[str(e[0])] = True
            
    f.write('module userModule \n')
    outgoing_edges = {}
    for e in edges_user:
        print(g[e[0]][e[1]]['prob_weight'] ) # use prob weight otherwise problems with unrolling; check why weight of 1...
        if e[0] not in outgoing_edges:
            outgoing_edges[e[0]] = [e]
        else:
            outgoing_edges[e[0]].append(e)
            
    for k in outgoing_edges:
        out_sum = sum([len(mapping[element]) for element in outgoing_edges[k]])
        print(out_sum)
    for e in edges_user:
        weight = 0
        if "label" in g[e[0]][e[1]]:
            weight = int(g[e[0]][e[1]]["label"])
        f.write('[] state='+ states[e[0]] + " & user -> (state'="+ states[e[1]]+" ) & (user'=false) & (gas' = gas + "+str(weight)+") & (steps' = steps+1)")
        if e[1] in terminal:
            f.write(" & (final_gas' = gas + "+str(weight)+")")
            if "pos" in e[1]:
                f.write(" & (positive' = true) ")
            else:
                f.write(" & (negative' = true) ")
        f.write("; \n")
    for s in states:
        if s in user_control and user_control[s]:
            if s in provider_control and provider_control[s]:
                f.write('[] state='+ states[s] + " & user -> (provider'=true) & (user'=false); \n")
    f.write('endmodule \n')
    
    f.write('module providerModule \n')
    for e in edges_provider:
        weight = 0
        if "label" in g[e[0]][e[1]]:
            weight = int(g[e[0]][e[1]]["label"])         
        f.write('[] state='+ states[e[0]] + " & provider -> (state'="+ states[e[1]]+" ) & (provider'=false) & (gas' = gas + "+str(weight)+") & (steps' = steps+1)")
        if e[1] in terminal:
            f.write(" & (final_gas' = gas + "+str(weight)+")")
            if "pos" in e[1]:
                f.write(" & (positive' = true) ")
            else:
                f.write(" & (negative' = true) ")
        f.write("; \n")
    f.write('endmodule \n')
    
    f.write('module controll \n')
    for s in states:
        if s in user_control and user_control[s] and s in provider_control and provider_control[s]:
            f.write("[] state="+ states[s] + " & !user & !provider -> q : (provider' = true) + (1-q) : (user' = true); \n")
        elif s in user_control and user_control[s]:
            f.write("[] state="+ states[s] + " & !user & !provider -> (user' = true); \n")
        elif s in provider_control and provider_control[s]:
            f.write("[] state="+ states[s] + " & !user & !provider -> (provider' = true); \n")
        else:
            print("no one controlls", s)
            assert(False or len(g.out_edges(s))==0)
            
    for s in terminal:
        f.write("[] state="+ states[s] + " & !user & !provider -> (state' = "+ str(states[s])+"); \n")
    f.write('endmodule \n')
    
terminal = [s for s in g_after_unroll if "neg" in s or "pos" in s]
print(terminal)
to_prism(g_after_unroll, "generated.prism", terminal, edge_mapping_after)

In [ ]:
def remove_undecision(g):
    g = copy.deepcopy(g)
    for s in g:
        uncontrollable = False
        controllable = False
        for e in g.edges(s):
            print(e)
            attr = g[e[0]][e[1]]
            if attr["controllable"]:
                controllable = True
            else:
                uncontrollable = True
        to_remove = []
        if controllable and uncontrollable:
            for e in g.edges(s):
                attr = g[e[0]][e[1]]
                if attr["controllable"]:
                    to_remove.append(e)
        for e in to_remove:
            g.remove_edge(e[0],e[1])
    return g
#g_decideable = remove_undecision(g)
#draw_dfg(g_decideable, 'decideable.ps', "dot")
#to_uppaal(g_decideable, "decideable.xml")

def query(g):
    # partial graph implications, per activity
    results = {}
    intervals = {}
    min_gas = {}
    #print(len(set([x['concept:name'] for trace in filtered_log for x in trace])))
    counter = 0
    for a in g.nodes:
        print("counter", counter, "current action", a)
        counter += 1
        states = [a]

        sub_nodes = set()
        for s in states:
            sub_nodes.update(set(list(nx.descendants(g, s))))
            sub_nodes.add(s)
        if len(sub_nodes) > 100:
            continue
        subgraph = nx.subgraph(g, sub_nodes)
        subgraph = nx.DiGraph(subgraph)

        # add start node to subgraph
        start_nodes = []
        for n in subgraph.nodes:
            if subgraph.in_degree(n) == 0:
                start_nodes.append(n)
        for n in start_nodes:
            subgraph.add_edge("start", n)
            subgraph["start"][n]["controllable"] = True
            subgraph["start"][n]["label"] = 0
        # if initial node lies on cycle, per default set as start node
        if "start" not in subgraph.nodes:
            for n in states:
                subgraph.add_edge("start", n)
                subgraph["start"][n]["controllable"] = True
                subgraph["start"][n]["label"] = 0

        target = [s for s in subgraph.nodes if "positive" in s or "negative" in s]
        subgraph_unrolled = unroll(subgraph, "start", target, 1)
        positives = []
        for s in subgraph_unrolled.nodes:
            if "positive" in s:
                positives.append(s)
        assert(len(positives) <= 1)
        to_uppaal(subgraph_unrolled, "bpi2017subgraph.xml")
        out = subprocess.Popen(["/home/paul/Downloads/uppaal-4.1.20-stratego-9-linux64/bin/verifyta", "/home/paul/Downloads/uppaal-4.1.20-stratego-9-linux64/bpi2017subgraph.xml", "/home/paul/Downloads/uppaal-4.1.20-stratego-9-linux64/queries/guaranteed.q"], stdout=subprocess.PIPE)
        result = "is satisfied" in str(out.communicate()[0])
        print("result", result)
        results[a] = result

        decideable = remove_undecision(subgraph)
        target = [s for s in decideable.nodes if "positive" in s or "negative" in s]
        decideable_unrolled = unroll(decideable, "start", target, 1)
        positives = []
        for s in decideable_unrolled.nodes:
            if "positive" in s:
                positives.append(s)
        assert(len(positives) <= 1)
        to_uppaal(decideable_unrolled, "bpi2017subgraph.xml")
        out = subprocess.Popen(["/home/paul/Downloads/uppaal-4.1.20-stratego-9-linux64/bin/verifyta", "/home/paul/Downloads/uppaal-4.1.20-stratego-9-linux64/bpi2017subgraph.xml", "/home/paul/Downloads/uppaal-4.1.20-stratego-9-linux64/queries/probability.q", "-E", "0.001"], stdout=subprocess.PIPE)
        s = str(out.communicate()[0]).split("formula 3")[1].split("Pr(<>")[1]
        interval = s[s.find("["):s.find("]")+1]
        print("interval", interval)
        intervals[a] = interval

        out = subprocess.Popen(["/home/paul/Downloads/uppaal-4.1.20-stratego-9-linux64/bin/verifyta", "/home/paul/Downloads/uppaal-4.1.20-stratego-9-linux64/bpi2017subgraph.xml", "/home/paul/Downloads/uppaal-4.1.20-stratego-9-linux64/queries/min_gas.q"], stdout=subprocess.PIPE)
        s = str(out.communicate()[0]).split("formula 3")[1].split("Values in")[1]
        interval = s[s.find("["):s.find("]")+1]
        print("min gas", interval)
        min_gas[a] = interval
        
    return results, intervals, min_gas

results_before, intervals_before, min_gas_before = query(g_before_colored)
results_after, intervals_after, min_gas_after = query(g_after_colored)

In [ ]:
events_before = set([x['concept:name'] for trace in filtered_log_before for x in trace])
events_after = set([x['concept:name'] for trace in filtered_log_after for x in trace])
events = events_before.union(events_after)

events_short = {}
for e in events:
    numbers = [int(s) for s in e.split(" ") if s.isdigit()]
    if not numbers:
        events_short[e] = e[:3]
    else:
        assert(len(numbers)==1)
        events_short[e] = e[:3]+str(numbers[0])
    if "online only" in e:
        events_short[e] += ".o"
    if "mail and online" in e:
        events_short[e] += ".m+o"
    if "SHORT" in e:
        events_short[e] += ".s"
    if "SUPER LONG" in e:
        events_short[e] += ".sl"
    elif "LONG" in e:
        events_short[e] += ".s"
    print(e, events_short[e])
print(len(events_short), set([e for e in events_short[e]]))
print(events)
print(len(events_before), len(events_after))

In [ ]:
print("before")
for s in g_before_colored:
    s = s.split("-")
    if "Call" in s[0] and "Call" in s[-1]:
        print(s)
        
print("")
for s in g_before_colored:
    s = s.split("-")
    if len(s) < 2:
        continue
    if "O_Returned" in s[1]:
        print(s)
        
print("after")        
for s in g_after_colored:
    s = s.split("-")
    if "Call" in s[0] and "Call" in s[-1]:
        print(s)

In [ ]:
def compute_labels(g1, g2):
    new_labels = {}
    new_labels_int = {}
    
    both_nodes = []
    for n in g1.nodes:
        if n not in both_nodes:
            both_nodes.append(n)
    for n in g2.nodes:
        if n not in both_nodes:
            both_nodes.append(n)
    
    #both_nodes = set(g1.nodes).union(set(g2.nodes))
    for n,i in zip(both_nodes, range(len(both_nodes))):
        h = [e.strip() for e in n.split("-")]
        h1 = events_short[h[0]]
        for e in h[1:]:
            h1 += " - "
            h1 += events_short[e]
        new_labels[n] = h1
        new_labels_int[n] = i

    new_labels.pop("start")
    #new_labels.pop("positive")
    #new_labels.pop("negative")
    
    new_labels_int.pop("start")
    #new_labels_int["positive"] = "pos"
    #new_labels_int["negative"] = "neg"
    return new_labels_int

def draw_dfg(g, name, results, intervals, labels, layout = "sfdp"):
    # build graph with variable thicknes
    #scaling = 1/np.mean(list(nx.get_edge_attributes(g,'edge_weight').values()))
    g = copy.deepcopy(g)
    
    for s in results:
        pos = False
        neg = False
        if s not in g:
            # after contraction
            continue
        for n in g[s]:
            if "pos" in n:
                pos = True
            if "neg" in n:
                neg = True
        if pos and neg:
            g.nodes[s]['color'] = "blue"
            print(s,labels[s])
        else:
            #n = g.get_node(s)
            #n.attr['fillcolor']="#CCCCFF"
            pass
            #g.nodes[s]['color'] = "grey"
            #g.node[s]['color']="grey"
        if intervals[s] not in ["[0,0.000999738]", "[0.999,1]"]:
            pass # probabilities not included
            #g.nodes[s]['shape'] = "doublecircle"
    count = 0
    for e in g.edges:
        if "TIMEOUT" in g[e[0]][e[1]]['action']:
            g[e[0]][e[1]]['color'] = "#9d4edd"
            count += 1
            #g[e[0]][e[1]]['style'] = "dashed"
    print("counted", count, "edges")
    
    edges = nx.bfs_edges(g, "start")
    nodes = ["start"] + [v for u, v in edges]   
    
    
    g = nx.relabel_nodes(g, labels)
    #g = nx.convert_node_labels_to_integers(g)
    
    # merge start cluster together
    for n in nx.descendants_at_distance(g, "start", 2).union(nx.descendants_at_distance(g, "start", 1)):
        g = nx.contracted_nodes(g, "start", n)
        
    g.remove_edges_from(nx.selfloop_edges(g))
    
    for s in g:
        outgoing_sum = 0
        if "color" in g.nodes[s]:
            if g.nodes[s]['color'] == "blue":
                    continue
        for n in g[s]:
            if "label" in g[s][n]:
                outgoing_sum += g[s][n]["label"]
        if outgoing_sum >= 16:
            g.nodes[s]['color'] = "#0ead69"
        if outgoing_sum <= -30:
            g.nodes[s]['color'] = "#9a031e"

    A = to_agraph(g)
    edge_weights = nx.get_edge_attributes(g,'edge_weight')
    for e in edge_weights:
        e = A.get_edge(e[0], e[1])
        #e.attr["penwidth"] = 40
        #e.attr["fontsize"] = 40
    for e in g.edges:
        edge = A.get_edge(e[0], e[1])
        if 'controllable' in g[e[0]][e[1]]:
            if not g[e[0]][e[1]]['controllable']:
                edge.attr["style"] = "dotted"
        edge.attr["label"] = ""
        #edge.attr["fontsize"] = 40
        #edge.attr["pennwidth"] = 40
                #edge.attr["label"] += " count " + str(g[e[0]][e[1]]["prob_weight"])
        #A.add_edge(e[0], e[1], penwidth = edge_weights[e]*scaling)
    for n in A:
        pass
        #print(str(new_labels[n]))
    
    for n in A.nodes():
        n.attr['fontsize'] = 45
        n.attr['penwidth'] = 10
        if n == "pos":
            n.attr['color'] = "#0ead69"
            n.attr['fontcolor'] = "#0ead69"
        if n == "neg":
            n.attr['color'] = "#9a031e"
            n.attr['fontcolor'] = "#9a031e"
        if n not in ["pos", "neg"]:
            n.attr['label'] = ""
            #n.attr['color'] = "grey"
        #print("bound", str(lower_bounds[n]))
        #n.attr['label'] = str(min_gas[n])
    for e in A.edges():
        e.attr['penwidth'] = 9
    
    #A.graph_attr.update(size="7.75,10.25")
    A.write(name.split(".")[0]+".dot")
    A.layout(layout)
    print("Plotted", name)
    A.draw(name)
    
    return g
    
def reachable_cluster(g, results, intervals, name, labels):
    pos_cluster = []
    neg_cluster = []
    g = copy.deepcopy(g)
    to_remove = []
    for s in g:
        subgraph = nx.subgraph(g, set(list(nx.descendants(g, s))))
        subgraph = nx.DiGraph(subgraph)
        nodes = [s for s in subgraph]
        sub_results = [results[n] for n in results if n in nodes]
        if len(set(sub_results))<2:
            if results[s]:
                pos_cluster.append(s)
            else:
                neg_cluster.append(s)

    for s in pos_cluster:
        g = nx.contracted_nodes(g, "pos", s)
    for s in neg_cluster:
        g = nx.contracted_nodes(g, "neg", s)
   
    g.remove_edges_from(nx.selfloop_edges(g))
    
    g = merge_connections(g)
    
    print("negative", len(g.in_edges("negative")))
    print("positive", len(g.in_edges("positive")))
    print("paths", len(list(nx.all_simple_paths(g, "start", "positive"))))
    
    return draw_dfg(g, name, results, intervals, labels, layout = "dot")

    
def get_connection(g):
    for s in g:
        if len(list(g[s]))!= 1:
            continue
        assert(len(list(g[s]))==1)
        v = list(g[s])[0]

        edges = list(g.in_edges(v))
        if len(edges) == 1:
            s1 = edges[0][0]
            v1 = edges[0][1]
            assert(s == s1 and v == v1)
            return (s,v)
    return None

def merge_connections(g):
    g = copy.deepcopy(g)
    
    edge = get_connection(g)
    while(edge != None):
        g = nx.contracted_nodes(g, edge[0], edge[1])
        g.remove_edges_from(nx.selfloop_edges(g))
        edge = get_connection(g)
        
    return g

labels = compute_labels(g_before_colored, g_after_colored)
before_reachable = reachable_cluster(g_before_colored, results_before, intervals_before, 'clustered_before.png', labels)
after_reachable = reachable_cluster(g_after_colored, results_after, intervals_after, 'clustered_after.png', labels)

In [ ]:
def pos_distance(g):
    count = 0
    length_sum = 0
    for n in g:
        if "neg" in g[n] and "TIMEOUT" in g[n]["neg"]['action']:
            length = nx.shortest_path_length(g, "start", n)
            length_sum += length
            count += 1
    print(count, length_sum, length_sum/count)
pos_distance(before_reachable)
pos_distance(after_reachable)

In [ ]:
print("before drift number edges to pos", len(before_reachable.in_edges("pos")))
print(len(before_reachable.in_edges("neg")))
print("after drift number edges to neg", len(after_reachable.in_edges("pos")))
print(len(after_reachable.in_edges("neg")))

count = set()
for s in before_reachable:
    if "color" in before_reachable.nodes[s]:
        if before_reachable.nodes[s]['color'] == "blue":
            count = count.union(set([x[0] for x in list(before_reachable.in_edges(s))]))
to_remove = []
for e in count:
    if "color" in before_reachable.nodes[e]:
        if before_reachable.nodes[e]['color'] == "blue":
            to_remove.append(e)
for e in to_remove:
    count.remove(e)
print(count)
print(len(count))

count = set()
for s in after_reachable:
    if "color" in after_reachable.nodes[s]:
        if after_reachable.nodes[s]['color'] == "blue":
            count = count.union(set([x[0] for x in list(after_reachable.in_edges(s))]))

to_remove = []
for e in count:
    if "color" in after_reachable.nodes[e]:
        if after_reachable.nodes[e]['color'] == "blue":
            to_remove.append(e)
for e in to_remove:
    count.remove(e)
print(count)
print(len(count))

print("159 is contained", 159 in list(before_reachable.nodes))
print("139 is contained", 139 in list(before_reachable.nodes))

In [ ]:
contained_nodes = {}
for n in before_reachable:
    edges = before_reachable[n]
    contained = True
    if n in ["negative", "positive", "start"]:
        contained_nodes[n] = False
        continue
    if n not in after_reachable:
        contained = False
    else:
        for e in before_reachable[n]:
            if (n,e) not in after_reachable.edges:
                contained = False
        for e in before_reachable.in_edges(n):
            if e not in after_reachable.edges:
                contained = False
    if contained:
        print(n)
    contained_nodes[n] = contained

g_test = copy.deepcopy(before_reachable)

nodes = [n for n in contained_nodes if contained_nodes[n]]
print(nodes)
subgraph = g_test.subgraph(nodes)
print(list(subgraph.nodes))
A = to_agraph(subgraph)
A.layout("dot")
A.draw("test_merge.png")

#for n in [160, 96, 19, 34, 71, 60, 46,93, 95, 29, 59, 118]:
for n in [160, 96, 19]:
    g_test = nx.contracted_nodes(g_test, "start", n)
A = to_agraph(g_test)
A.layout("dot")
A.draw("test_merge.png")

In [ ]:
G=nx.DiGraph()
# example 1
#G.add_nodes_from(["1","2","3","4","5"])
#G.add_edges_from([("1","2"),("2","3"),("3","2"),("2","4"),("4","2"),("2","5")])
#target = "5"

# example 2
#G.add_nodes_from(["1","2","3","4","5"])
#G.add_edges_from([("1","2"),("1","3"),("3","2"),("2", "3"),("2","4"),("4","3"),("4","5")])
#target = "5"

# example 3
#G.add_nodes_from(["1","2","3","4"])
#G.add_edges_from([("1","2"),("2","3"),("2","4"),("4", "2")])
#target = "3"

# example 4
G.add_nodes_from(["1","2","3","4"])
G.add_edges_from([("1","2"),("1","3"),("2","3"),("3", "2"), ("2", "4")])
target = "4"

pos = nx.planar_layout(G)
nx.draw(G, with_labels = True, pos = pos)

k = 1
start = "1"

G_gen = unroll(G, start,[target],1)
A = to_agraph(G_gen)
A.layout('dot', args='-Grankdir=LR')
A.draw('gen.png')
A